In [ ]:
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from utils.image_manager import ImagesManager
from utils.mask_manager import MaskManager
from quickshift_tabular.quickshift_tabular import quickshift_tab

In [ ]:
path_to_images='./data/debug'
images_extension = 'jpg'
classifier = keras.models.load_model('./model/model.h5')

In [ ]:
# Pretrained model
pretrained_model = ResNet50(weights='imagenet', include_top=True)
output_interest = pretrained_model.layers[-2].output
model = keras.models.Model(inputs=pretrained_model.inputs, outputs=output_interest)

In [ ]:
for layer in model.layers:
    layer.trainable = False

In [ ]:
import os
from skimage import io
from skimage.measure import EllipseModel
import numpy as np
import pandas as pd
from functools import partial

class FeaturesManager:
    
    def __init__(self, img, image_name, mask):

        self.img = img
        self.image_name = image_name
        self.mask = mask
        self.features_properties = [
            {'name': 'image_name', 'multiple_values': False, 'is_index': True, 'fun': self.get_image_name},
            {'name': 'num_superpixel', 'multiple_values': False, 'is_index': True, 'fun': self.get_num_superpixel},
            {'name': 'total_pixels', 'multiple_values': False, 'is_index': False, 'fun': self.get_total_pixels},
            {'name': 'mean_channel', 'multiple_values': True, 'is_index': False, 'fun': self.get_mean},
            {'name': 'std_channel', 'multiple_values': True, 'is_index': False, 'fun': self.get_std},
            {'name': 'channels_correlation', 'multiple_values': True, 'is_index': False, 'fun': self.get_channels_correlation},
            {'name': 'mass_center', 'multiple_values': True, 'is_index': False, 'fun': self.get_mass_center},
            {'name': 'ellipse_ratio', 'multiple_values': False, 'is_index': False, 'fun': self.get_ellipse_features},
            
            #{'name': 'q_5_channel', 'multiple_values': True, 'is_index': False, 'fun': partial(self.get_quant, q=.05, per_axis=True)},
            #{'name': 'q_10_channel', 'multiple_values': True, 'is_index': False, 'fun': partial(self.get_quant, q=.1, per_axis=True)},
            #{'name': 'q_25_channel', 'multiple_values': True, 'is_index': False, 'fun': partial(self.get_quant, q=.25, per_axis=True)},
            #{'name': 'q_50_channel', 'multiple_values': True, 'is_index': False, 'fun': partial(self.get_quant, q=.5, per_axis=True)},
            #{'name': 'q_75_channel', 'multiple_values': True, 'is_index': False, 'fun': partial(self.get_quant, q=.75, per_axis=True)},
            #{'name': 'q_90_channel', 'multiple_values': True, 'is_index': False, 'fun': partial(self.get_quant, q=.9, per_axis=True)},
            #{'name': 'q_95_channel', 'multiple_values': True, 'is_index': False, 'fun': partial(self.get_quant, q=.95, per_axis=True)},
            
            {'name': 'q_5', 'multiple_values': False, 'is_index': False, 'fun': partial(self.get_quant, q=.05, per_axis=False)},
            {'name': 'q_10', 'multiple_values': False, 'is_index': False, 'fun': partial(self.get_quant, q=.1, per_axis=False)},
            {'name': 'q_25', 'multiple_values': False, 'is_index': False, 'fun': partial(self.get_quant, q=.25, per_axis=False)},
            {'name': 'q_50', 'multiple_values': False, 'is_index': False, 'fun': partial(self.get_quant, q=.5, per_axis=False)},
            {'name': 'q_75', 'multiple_values': False, 'is_index': False, 'fun': partial(self.get_quant, q=.75, per_axis=False)},
            {'name': 'q_90', 'multiple_values': False, 'is_index': False, 'fun': partial(self.get_quant, q=.9, per_axis=False)},
            {'name': 'q_95', 'multiple_values': False, 'is_index': False, 'fun': partial(self.get_quant, q=.95, per_axis=False)},
        ]
        #self.normalise_features = normalise_features
        #self.num_channels = num_channels
        self.mask_shape = mask.shape
        
    def get_quant(self, q, superpixel, per_axis, normalise_features, *args, **kwargs):
        if normalise_features:
            if per_axis:
                return np.quantile(a=superpixel/255, q=q, axis=0)
            else:
                return np.quantile(a=superpixel/255, q=q)
        else:
            if per_axis:
                return np.quantile(a=superpixel, q=q, axis=0)
            else:
                return np.quantile(a=superpixel, q=q)
                
        
    def get_num_superpixel(self, num_superpixel, *args, **kwargs):
        return int(num_superpixel)
    
    def get_total_pixels(self, superpixel, normalise_features, *args, **kwargs):
        total_pixels = self.mask_shape[0] * self.mask_shape[1]
        if normalise_features:
            return superpixel.shape[0]/total_pixels
        else:
            return superpixel.shape[0]
    
    def get_image_name(self, *args, **kwargs):
        return self.image_name
    
    def get_mean(self, superpixel, normalise_features, *args, **kwargs):
        if normalise_features:
            superpixels_norm = superpixel/255
            return np.mean(superpixels_norm, axis=0)
        else:
            return np.mean(superpixel, axis=0)
    
    def get_std(self, superpixel, normalise_features, *args, **kwargs):
        if normalise_features:
            return np.std(superpixel/255, axis=0)
        else:
            return np.std(superpixel, axis=0)
        
    def get_channels_correlation(self, superpixel, normalise_features, num_channels, *args, **kwargs):
        
        superpixel_norm = superpixel
        superpixel_shape = superpixel_norm.shape
        
        if normalise_features:
            superpixel_norm = superpixel/255
        
        if superpixel_shape[0]>1 and superpixel_shape[1]>1:
            corr_matrix = np.corrcoef(superpixel_norm, rowvar=False)
            idx = np.triu_indices(n=num_channels, m=num_channels, k=1)
            corr_vector = corr_matrix[idx]
        
        else:
            n = num_channels
            len_vector = int(n*(n-1)/2)
            corr_vector = np.repeat(None, len_vector)
            
        return corr_vector
    
    def get_mass_center(self, coordinates, normalise_features, *args, **kwargs):
        
        coordinates_norm = np.asarray(coordinates, dtype=np.float64)
        if normalise_features:
            for i,len_axis in enumerate(self.mask_shape):
                coordinates_norm[i] = coordinates_norm[i]/len_axis
        return np.mean(coordinates_norm, axis=1)
        
    def get_ellipse_features(self, coordinates, *args, **kwargs):
        
        if len(coordinates[0]) > 1 and len(coordinates[1]) > 1:
            coordinates_stack = np.column_stack(coordinates)
            ellipse = EllipseModel()
            ellipse.estimate(coordinates_stack)
            xc, yc, a, b, theta = ellipse.params
            ratio = b/a
            if(b>a):
                ratio = 1/ratio
        else:
            ratio = None
        
        return ratio

    def create_data_frame(self, dict_features):
        
        df = pd.DataFrame(dict_features)
        idx_columns = []
            
        for feature in self.features_properties:
            if feature['multiple_values']:
                feature_name = feature['name']
                values = df[feature_name].to_list()
                len_values = len(values[0])
                names = [feature_name + str(i) for i in range(len_values)]
                df[names] = values
                df.drop(columns=feature_name, inplace=True)
                
            if feature['is_index']:
                idx_columns.append(feature['name'])
        
        if len(idx_columns)>0:
            df.set_index(idx_columns, inplace=True)

        return df
    
    def clean_data(self, df):

        columns = df.columns
        for column in columns:
            values = df[column]
            mean_column = np.mean(values)
            idx_missing = np.isnan(values)

            if len(idx_missing)>0:
                df.loc[idx_missing, column] = mean_column        
    
    def extract_from_manual(self, normalise_features, num_channels):

        img = self.img
        mask = self.mask

        deleted_superpixels = []
        dict_features = {}
        
        # Obtain features
        unique_superpixels = np.unique(self.mask)
        
        for index in unique_superpixels:
            coordinates = np.where(mask == index)
            superpixel = img[coordinates]
            num_pixels = self.get_total_pixels(superpixel=superpixel,
                                               normalise_features=normalise_features)
            
            if num_pixels<=1:
                deleted_superpixels.append(index)
            
            for feature in self.features_properties:
                feature_name = feature['name']
                feature_values = feature['fun'](superpixel=superpixel, num_superpixel=index,
                                                img=img, mask=mask, coordinates=coordinates,
                                                normalise_features=normalise_features,
                                                num_channels=num_channels)

                if feature_name in dict_features.keys():
                    dict_features[feature_name].append(feature_values)
                else:
                    dict_features[feature_name] = [feature_values]
        
        df_features = self.create_data_frame(dict_features)
        
        self.clean_data(df_features)
        
        return df_features, deleted_superpixels
    
    def get_smalles_patch(self, num_superpixel):
        
        img = self.img
        mask = self.mask
        idx_mask = np.where(mask == num_superpixel)
        min_idx = np.min(idx_mask, axis=-1)
        max_idx = np.max(idx_mask, axis=-1)
        slicer = []

        for min_val, max_val in zip(min_idx, max_idx):
            slice_dim = slice(min_val, max_val+1, 1)
            slicer.append(slice_dim)

        slicer = tuple(slicer)
        sliced_image = img[slicer]
        return img[slicer]

    def extract_from_model(self, model_feature_extractor, target_shape):
        img = self.img
        mask = self.mask
        list_features = []
        list_num_superpixel = []
        list_image_name = []
        list_superpixels = np.unique(mask)
        list_mass_center = []
        list_total_zeros = []
        for sp in list_superpixels:
            coordinates = np.where(mask == sp)
            mass_center = self.get_mass_center(coordinates=coordinates, normalise_features=True)
            smallest_patch = self.get_smalles_patch(num_superpixel=sp)
            smallest_patch = image.smart_resize(smallest_patch, size=target_shape)
            smallest_patch = np.expand_dims(smallest_patch, axis=0)
            features = model_feature_extractor(smallest_patch)
            np_features = features.numpy()
            np_features = np_features[0]
            total_zeros = np.sum(np_features <= 10**(-10))
            list_total_zeros.append(total_zeros)
            list_features.append(np_features)
            list_num_superpixel.append(sp)
            list_image_name.append(self.image_name)
            list_mass_center.append(mass_center.tolist())
        columns = ['mass_center{}'.format(i) for i in range(len(mass_center.tolist()))]
        df_mass_center = pd.DataFrame(np.array(list_mass_center), columns=columns)
        df_features_model = pd.DataFrame(np.array(list_features))
        df_features = pd.concat([df_mass_center, df_features_model], axis=1)
        df_features['image_name'] = list_image_name
        df_features['num_superpixel'] = list_num_superpixel
        df_features.set_index(['image_name', 'num_superpixel'], inplace=True)
        return df_features, list_total_zeros

    def get_features(self, use_model_to_extract_features, 
                     model_feature_extractor, target_shape,
                     normalise_features, num_channels):

        if use_model_to_extract_features:
            df, list_total_zeros = self.extract_from_model(model_feature_extractor, target_shape)
            deleted_superpixels = []
        else:
            list_total_zeros = []
            df, deleted_superpixels = self.extract_from_manual(normalise_features=normalise_features,
                                                               num_channels=num_channels)
        return df, deleted_superpixels, list_total_zeros


In [ ]:
def get_superpixels_information(path_to_images, images_extension,
                                normalise_features=True, num_channels=3,
                                kernel_size=4, max_dist=20, ratio=.2,
                                kernel_size_tab=1, max_dist_tab=0.3, ratio_tab=0.5,
                                num_working_examples=None,
                                use_model_to_extract_features=False,
                                model_feature_extractor=None, target_shape=None,
                                verbose=True, want_to_plot_clusters=False):

    # Get images name
    images_manager = ImagesManager(location=path_to_images, images_extension=images_extension)
    images_names = images_manager.get_images_names()
    
    # Select the number of examples
    if num_working_examples is not None:
        images_to_take = num_working_examples
        if images_to_take>len(images_names):
            images_to_take = len(images_names)
        images_names = images_names[:images_to_take]

    dict_masks = {}
    dict_deleted_superpixels = {}
    list_num_superpixels = []
    list_df = []
    spatial_columns = ['mass_center0', 'mass_center1']

    for image_name in images_names:
        # Get the image
        img = images_manager.get_image(image_name)

        # Get the mask
        masks_manager = MaskManager()
        mask = masks_manager.get_mask(img=img, kernel_size=kernel_size, max_dist=max_dist, ratio=ratio)
        num_superpixels_bf = np.unique(mask)
        if verbose:
            print('---------------------------------------------------------')
            print('Working on image:', image_name)
            print('\tNum superpixels before tabular quickshift:', len(num_superpixels_bf))

        #Create features for each superpixel
        features_manager = FeaturesManager(img=img, image_name=image_name, mask=mask)
        df_features, _, list_total_zeros = features_manager.get_features(use_model_to_extract_features=use_model_to_extract_features,
                                                                         model_feature_extractor=model_feature_extractor,
                                                                         target_shape=target_shape,
                                                                         normalise_features=normalise_features,
                                                                         num_channels=num_channels)

        # Print number of features equal to 0
        if verbose and use_model_to_extract_features:
            print('\tTotal features equal to 0:', int(np.mean(list_total_zeros)))
         
        # Use tabluar quickshift to cluster some superpixels
        col_names = df_features.columns.tolist()
        col_reodered = spatial_columns + [x for x in col_names if x not in spatial_columns]
        df_features = df_features.reindex(columns=col_reodered)
        mask_list = [x[1] for x in df_features.index.tolist()]
        X = df_features.to_numpy()
        mask_tabular_quickshift_list = quickshift_tab(X=X, ratio=ratio_tab,
                                                      kernel_size=kernel_size_tab, max_dist=max_dist_tab)
        mask_tabluar_quickshift = np.empty(shape=mask.shape)
        
        if verbose:
            print('\tNum superpixels after tabular quickshift:', len(np.unique(mask_tabular_quickshift_list)))
            
        # Create the new mask after using tabular quickshit
        for original_value, new_value in zip(mask_list, mask_tabular_quickshift_list):
            idx = np.where(mask == original_value)
            mask_tabluar_quickshift[idx] = new_value
        dict_masks[image_name] = mask_tabluar_quickshift
        list_num_superpixels.append(len(np.unique(mask_tabular_quickshift_list)))

        if verbose:
            plt.imshow(img)
            plt.show()
            plt.imshow(mask_tabluar_quickshift)
            plt.show()
            
            if want_to_plot_clusters:
                for sp_value in np.unique(mask_tabular_quickshift_list):
                    mk = mask_tabluar_quickshift == sp_value
                    mk = mk[..., np.newaxis]

                    img_occluded = img * mk
                    plt.imshow(img_occluded, vmin=np.min(img_occluded), vmax=np.max(img_occluded))
                    plt.show()
            
        
            

In [ ]:
data = get_superpixels_information(path_to_images=path_to_images, images_extension=images_extension,
                                   normalise_features=True, num_channels=3,
                                   kernel_size=4, max_dist=20, ratio=.2,
                                   kernel_size_tab=1, max_dist_tab=12, ratio_tab=0.5,
                                   num_working_examples=1,
                                   use_model_to_extract_features=True, model_feature_extractor=model,
                                   target_shape=(224, 224),
                                   verbose=True, want_to_plot_clusters=True)